# Preprocessing Enron Emails

## Tasks

1. Import emails
2. Parse them to email format
3. Keep only original emails
4. Detect chain emails
4. Remove html noise
5. Divide dataset for only chains with replies. No forwards nor noise
5. Export emails to pickle file

In [1]:
import email
import pickle as pkl
import re

import numpy as np
import pandas as pd

from collections import Counter
from datetime import datetime, timedelta
from dateutil.parser import parse
from pathlib import Path

## Extraction

In [2]:
email_dir = Path(Path.cwd().parent, Path('data/raw/maildir'))
data_dir = Path(Path.cwd().parent, Path('data/interim'))


def parse_emails(path):
    with open(path, 'r', encoding='windows-1252') as f:
        parsed_email = email.message_from_file(f)
    return parsed_email


def get_parsed_emails(paths):
    emails = []
    for i, path in enumerate(paths):
        eml = parse_emails(path)
        tms = int(parse(eml['Date']).timestamp())
        emails.append((i, eml, tms))
    return emails


def get_parsed_emails(paths, dic=None):
    for path in paths:
        eml = parse_emails(path)
        temp = {k:v for k, v in eml.items() + [('Message', eml.get_payload()), ('Timestamp', int(parse(eml['Date']).timestamp()))]}
        for k,v in dic.items():
            dic[k].append(temp.get(k))
    return dic


def remove_spaces(string):
    if string is not None:        
        string = re.sub('\s+', ' ', string)
        string = string.split(', ')
    return string


def get_chain(data):
    df_ = data.loc[:, ['key', 'Timestamp', 'Reply']].sort_values(by=['key', 'Reply', 'Timestamp'])
    chains = {}
    counter = 0
    for idx, row in df_.iterrows():
        key = f"{row['key']}_{counter:03d}"
        if key not in chains:
            counter = 0
            key = f"{row['key']}_{counter:03d}"
            chains[key] = {'length': 1, 'email_ids': [idx]}
        else:
            if row['Reply']:
                chains[key]['length'] += 1
                chains[key]['email_ids'].append(idx)
            else:
                counter += 1
                key = f"{row['key']}_{counter:03d}"
                chains[key] = {'length': 1, 'email_ids': [idx]}

    chains_new = {}
    for i, row in enumerate(chains):
        chains_new[i] = {'chain_id': row} | chains[row]

    return chains_new


def get_longest_chain(chain):
    MAX = 0
    longest_chain = []
    for k, v in chain.items():
        if v['length'] > MAX:
            MAX = v['length']
            longest_chain = (k, v['length'], v['email_ids'])
        elif v['length'] == MAX:
            if isinstance(longest_chain, tuple):
                longest_chain = [longest_chain, (k, v['length'] ,v['email_ids'])]
            else:
                longest_chain.append((k, v['length'] ,v['email_ids']))
        
    return longest_chain


def assign_chain_id(data, chain):
    data.loc[:,'Chain'] = None
    data.loc[:,'Chain_len'] = None
    for k,v in chain.items():
        data.loc[v['email_ids'], 'Chain'] = k
        data.loc[v['email_ids'], 'Chain_len'] = int(v['length'])

In [3]:
%%time
parsed_emails_file = Path(data_dir, 'parsed_emails_chains_all.pkl')

if not parsed_emails_file.is_file():
    clean_emails = []
    for path in email_dir.rglob('*.'):
        if 'all_documents' not in str(path.parent) and 'discussion_threads' not in str(path.parent):
            clean_emails.append(path)
    email_dict = {
        'Message-ID': [],
        'Date': [],
        'From': [],
        'To': [],
        'Subject': [],
        'Cc': [],
        'Mime-Version': [],
        'Content-Type': [],
        'Content-Transfer-Encoding': [],
        'Bcc': [],
        'X-From': [],
        'X-To': [],
        'X-cc': [],
        'X-bcc': [],
        'X-Folder': [],
        'X-Origin': [],
        'X-FileName': [],
        'Message': [],
        'Timestamp': []
    }
    df = pd.DataFrame(get_parsed_emails(clean_emails, email_dict))
else:
    df = pd.read_pickle(parsed_emails_file)
    
    


CPU times: user 193 ms, sys: 257 ms, total: 449 ms
Wall time: 1.37 s


## Transformation


In [41]:
%%time
if not parsed_emails_file.is_file():
    df = df.loc[:, ['Subject', 'From', 'Message','To','Cc','Bcc','Timestamp']].drop_duplicates()
    df = df.sort_values(by='Timestamp').reset_index()
    df['to'] = df['To'].apply(lambda x: [] if x is None else remove_spaces(x))
    df['cc'] = df['Cc'].apply(lambda x: [] if x is None else remove_spaces(x))
    df['bcc'] = df['Bcc'].apply(lambda x: [] if x is None else remove_spaces(x))
    df['Sender'] = df['From']
    df['Recipients'] = df['to'] + df['cc'] + df['bcc']
    df['users'] = df['Sender'].apply(lambda x: [x]) + df['Recipients']
    df['users'] = df['users'].apply(lambda x: sorted(x))
    df['subject'] = df['Subject'].str.replace('(re\W*:|FW\W+|FWD\W*:)', '', regex=True, flags=re.IGNORECASE).str.strip()
    df['subject'] = df['subject'].apply(lambda x: ['None'] if x == '' else [x])
    df['key'] = df['subject'] + df['users']
    df['key'] = df['key'].agg('_'.join)
    df['Recipients'] = df['Recipients'].apply(lambda x: ', '.join(x))
    msg_end_pattern = re.compile('_{4,}.*|\n{3,}|<[^>]*>|-{4,}(.*)(\d{2}:\d{2}:\d{2})\s*(PM|AM)', re.MULTILINE)
    df['Message'] = df['Message'].str.replace(msg_end_pattern, '', regex=True)
    df['Message'] = df['Message'].str.replace('(?=-----Original Message-----)(?s)(.*$)', '', regex=True)
    df['Message'] = df['Message'].replace(r'^\s*$', np.nan, regex=True)
    df['Reply'] = False
    df.loc[df['Subject'].str.contains('(re\W*:)', flags=re.IGNORECASE, regex=True),'Reply'] = True
    chain_rlx = get_chain(df)
    assign_chain_id(df, chain_rlx)
    columns = ['Message', 'Reply', 'Chain', 'Chain_len', 'Subject', 'Sender', 'Recipients', 'Timestamp']
    df = df.loc[:, columns]

<timed exec>:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.


CPU times: user 2min 4s, sys: 985 ms, total: 2min 4s
Wall time: 2min 3s


In [4]:
emails_file = Path(data_dir, 'parsed_emails_chains_replies.pkl')
if not emails_file.is_file():
    chains_reply = df.loc[df['Reply'], 'Chain'].dropna().unique()
    df_re = df.loc[df['Chain'].isin(chains_reply)]
else:
    df_re = pd.read_pickle(emails_file)

## Loading


In [43]:
if not parsed_emails_file.is_file():    
    df.to_pickle(parsed_emails_file)

if not emails_file.is_file():
    df_re.to_pickle(emails_file)

In [7]:
df

,Message,Reply,Chain,Chain_len,Subject,Sender,Recipients,Timestamp
0,Thank you for signing up with eThink. You may...,False,205324,1,Your eThink Password,ethink@enron.com,elizabeth.sager@enron.com,315532800
1,Restructuring Today quotes Jeff at length on r...,True,130278,1,Re:,steven.kean@enron.com,kmagruder@newpower.com,315532800
2,Mary- Ken has been asked by Mayor Brown to cha...,True,188148,1,Re: Trains - Light rail,steven.kean@enron.com,mary.clark@enron.com,315532800
3,Attached are FERC reports,False,131869,1,,steven.kean@enron.com,,315532800
4,Enron Metals Re-Branding (created by Lauren U...,False,15042,1,Archive Log for 7/10/2001 2:43:30 PM,archiving@enron.com,,315532800
...,...,...,...,...,...,...,...,...
251063,\n\nUntitled Document\n\n\n\n\nEducate yoursel...,False,1609,1,(None),pse6yl706@aloha.net,,1716720597
251064,\nHappy New Year !\n - marks.xls,False,213847,1,marks.xls,cramer@cadvision.com,"linsider.jed@enron.com, john.zufferli@enron.co...",2334944052
251065,\nfeb dec trades 37.5 \nfeb dec LL went out 20...,False,218943,1,trades jan 2002,cramer@cadvision.com,john.zufferli@enron.com,2335391160
251066,"\nBOM 5th to 31st traded 34, 33.5 , 33.5 and ...",False,187755,2,Trades,cramer@cadvision.com,john.zufferli@enron.com,2335539586


In [50]:
df[df['Chain_len'] > 1].to_pickle(Path(data_dir, f'parsed_emails_chains_gt_1.pkl'))

In [55]:
df_ch1 = df[df['Chain_len'] == 1]

In [56]:
n = 55_000
for g, d in df_ch1.groupby(np.arange(len(df_ch1)) // n):
    d.to_pickle(Path(data_dir, f'parsed_emails_chains_split_{g}.pkl'))

In [28]:
df[df['Chain_len'] > 9].dropna().to_pickle(Path(data_dir, f'parsed_emails_chains_ge_10.pkl'))

In [29]:
df[df['Chain_len'] == 3].dropna().to_pickle(Path(data_dir, f'parsed_emails_chains_eq_3.pkl'))

In [30]:
df[df['Chain_len'] == 2].dropna().to_pickle(Path(data_dir, f'parsed_emails_chains_eq_2.pkl'))

In [27]:
df[df['Chain_len'] == 4].dropna()

,Message,Reply,Chain,Chain_len,Subject,Sender,Recipients,Timestamp
302,Are you guys around this weekend? Any particu...,False,199022,4,Weekend,mark.taylor@enron.com,marc.r.cutler@bankamerica.com,918222960
303,I'm flying solo this weekend. No particular p...,True,199022,4,Re: Weekend,mark.taylor@enron.com,marc.r.cutler@bankamerica.com,918225720
305,"Happy hour with staff, not family :-(",True,199022,4,Re: Weekend,mark.taylor@enron.com,marc.r.cutler@bankamerica.com,918234000
750,I'm not sure whether to feel sorry for you or ...,True,199022,4,Re: Weekend,mark.taylor@enron.com,marc.r.cutler@bankamerica.com,928263000
973,Pay it.,True,92673,4,Re: Interfert,richard.sanders@enron.com,britt.davis@enron.com,930131820
...,...,...,...,...,...,...,...,...
250841,thanks,True,77446,4,RE: GISB Base Contract - Occidental Energy Mar...,robin.barbe@enron.com,"christina.finelli@enron.com, chip.schneider@en...",1025009958
250871,"i am thinking 87-90 on DIA, however you manage...",True,203187,4,RE: YOU CAN THANK ME LATER,joe.parks@enron.com,"'fenner@enron.com, chet_fenner@bmc.com",1025034934
250873,i figure by next week it should be good and so...,True,203187,4,RE: YOU CAN THANK ME LATER,joe.parks@enron.com,"'fenner@enron.com, chet_fenner@bmc.com",1025035252
250876,its called liquidation\n\n,True,203187,4,RE: YOU CAN THANK ME LATER,joe.parks@enron.com,"'fenner@enron.com, chet_fenner@bmc.com",1025035715


In [26]:
df[(df['Chain_len'] > 1) & (df['Chain_len'] < 5)].dropna()

,Message,Reply,Chain,Chain_len,Subject,Sender,Recipients,Timestamp
142,"\nHey Paul, how is it going?? Attached you'll...",False,87415,3,How are you?,educanto@msn.com,d..thomas@enron.com,883935960
144,Wish we could go - but we're off to Ft. Lauder...,True,75963,2,Re: Friday,mark.taylor@enron.com,marc.r.cutler@bankamerica.com,909762180
145,Hey Marc - any chance you guys might like to j...,True,75963,2,Re: Friday,mark.taylor@enron.com,marc.r.cutler@bankamerica.com,909762960
157,This message was returned to me - it looks lik...,False,192107,2,Undeliverable message,mark.taylor@enron.com,per.sekse@enron.com,910973340
158,I think you can go straight to performance rev...,True,192107,2,Re: Undeliverable message,mark.taylor@enron.com,per.sekse@enron.com,911468460
...,...,...,...,...,...,...,...,...
250917,"Jeff,\n\nThe files are in DesertSkyCurtail in ...",False,107371,2,May curtailment spread,mark.fisher@enron.com,jeff.duff@enron.com,1025701860
250920,"Mark,\n\nI checked the sums with what was sent...",True,107371,2,Re: May curtailment spread,jeff.duff@enron.com,mark.fisher@enron.com,1025704560
250970,"Tim,\n\nIn Oct 2001 I produced the attached re...",False,74063,2,Fluvanna and Trew Ranch reports,mark.fisher@enron.com,"tim.derrick@enron.com, jeff.duff@enron.com, je...",1026400320
250971,"Thanks. I will use this report, and we should...",True,74063,2,Re: Fluvanna and Trew Ranch reports,tim.derrick@enron.com,"mark.fisher@enron.com, jeff.duff@enron.com, je...",1026403800


In [33]:
df[(df['Chain_len'] > 3) & (df['Chain_len'] < 10)].dropna().to_pickle(Path(data_dir, f'parsed_emails_chains_ge_4_lt_10.pkl'))

In [31]:
df[(df['Chain_len'] > 3) & (df['Chain_len'] < 10)].dropna()

,Message,Reply,Chain,Chain_len,Subject,Sender,Recipients,Timestamp
302,Are you guys around this weekend? Any particu...,False,199022,4,Weekend,mark.taylor@enron.com,marc.r.cutler@bankamerica.com,918222960
303,I'm flying solo this weekend. No particular p...,True,199022,4,Re: Weekend,mark.taylor@enron.com,marc.r.cutler@bankamerica.com,918225720
305,"Happy hour with staff, not family :-(",True,199022,4,Re: Weekend,mark.taylor@enron.com,marc.r.cutler@bankamerica.com,918234000
462,not a thing yet,True,69384,7,Re: Exxon,elizabeth.sager@enron.com,john.malowney@enron.com,926337960
604,sorry to say but I haven't heard a thing,True,69384,7,Re: Exxon,elizabeth.sager@enron.com,john.malowney@enron.com,927200280
...,...,...,...,...,...,...,...,...
250874,Good point. That will be good time to come in...,True,203188,6,RE: YOU CAN THANK ME LATER,chet_fenner@bmc.com,joe.parks@enron.com,1025035400
250875,"Wooo, what a day! Blood-red screen, except fo...",True,203188,6,RE: YOU CAN THANK ME LATER,chet_fenner@bmc.com,joe.parks@enron.com,1025035638
250876,its called liquidation\n\n,True,203187,4,RE: YOU CAN THANK ME LATER,joe.parks@enron.com,"'fenner@enron.com, chet_fenner@bmc.com",1025035715
250879,"Si, Se?or Paras!\n\n \n\n",True,203188,6,RE: YOU CAN THANK ME LATER,chet_fenner@bmc.com,joe.parks@enron.com,1025036494
